In [ ]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

In [ ]:
MODEL_NAME="unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit"

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length=2048
lora_rank=64

model, tokeniser= FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=max_seq_length,
    max_lora_rank=lora_rank,
    load_in_4bit=False
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 11-29 19:14:20 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Gemma3N patching. Transformers: 4.56.2. vLLM: 0.10.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

Unsloth: Making `model.base_model.model.model.language_model` require gradients


In [ ]:
from datasets import load_dataset

dataset=load_dataset("openai/gsm8k","main",split="train")
dataset

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer'],
    num_rows: 7473
})

In [ ]:
dataset[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}

In [ ]:
def extract_hash_answer(text):
  if "####" not in text: return None
  return text.split("####")[1].strip()

extract_hash_answer(dataset[0]["answer"])

'72'

In [ ]:
reasoning_start = "<start_working_out>"
reasoning_end   = "<end_working_out>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

In [ ]:
def sys_prompt(reasoning_start="<start_working_out>", reasoning_end="<end_working_out>", solution_start="<SOLUTION>", solution_end="</SOLUTION>"):
  return f"""You are given a problem.
  Think about the problem and provide your working out.
  Place it between {reasoning_start}{reasoning_end}
  Then provide the solution.
  Place it between {solution_start}{solution_end}"""

sys_prompt()


'You are given a problem.\n  Think about the problem and provide your working out.\n  Place it between <start_working_out><end_working_out>\n  Then provide the solution.\n  Place it between <SOLUTION></SOLUTION>'

In [ ]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "user", "content": [{"type": "text", "text": sys_prompt()+"\n"+x["question"]}]},
    ],
    "answer": extract_hash_answer(x["answer"]),
})
dataset[0]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': '72',
 'prompt': [{'content': [{'text': 'You are given a problem.\n  Think about the problem and provide your working out.\n  Place it between <start_working_out><end_working_out>\n  Then provide the solution.\n  Place it between <SOLUTION></SOLUTION>\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
     'type': 'text'}],
   'role': 'user'}]}

In [ ]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

In [ ]:
match_format.search(
    "<start_working_out>Let me think!<end_working_out>"\
    "<SOLUTION>2</SOLUTION>",
)

<re.Match object; span=(0, 71), match='<start_working_out>Let me think!<end_working_out>>

In [ ]:
def match_format_exactly(completions,**kwargs):
  scores=[]
  for completion in completions:
    score=0
    content=completion[0]["content"]
    if match_format.search(content) is not None:
      score+=3.0
    scores.append(score)
  return scores

In [ ]:
def match_format_partially(completions, **kwargs):
  scores=[]
  for completion in completions:
    score=0
    content=completion[0]["content"]
    score+=0.5 if content.count(reasoning_start) == 1 else -1.0
    score+=0.5 if content.count(reasoning_end) == 1 else -1.0
    score+=0.5 if content.count(solution_start) == 1 else -1.0
    score+=0.5 if content.count(solution_end) == 1 else -1.0

    scores.append(score)
  return scores

In [ ]:
def extract_text(content_blocks):
    """Given content=[{"type":"text","text":...}, ...], return concatenated text."""
    return "".join(
        block["text"]
        for block in content_blocks
        if block.get("type") == "text"
    )


def check_answer(prompts, completions, answer, **kwargs):
    # Extract the text from the last user message (prompt)
    question_blocks = prompts[0][-1]["content"]
    question = extract_text(question_blocks)

    # Extract text from each completion
    responses = [
        completion[0]["content"]
        for completion in completions
    ]

    # Apply your regex template format
    extracted_responses=[]

    for r in responses:
      if(match_numbers.search(r) is not None):
        guess = match_numbers.search(r)
        extracted_responses.append(guess.group(1))
      else:
        extracted_responses.append(None)

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0

        if guess is None:
            scores.append(0)
            continue

        # Exact match
        if guess == true_answer:
            score += 3.0

        # Trimmed exact match
        elif guess.strip() == true_answer.strip():
            score += 3.0

        # Numeric closeness scoring
        else:
            try:
                ratio = float(guess) / float(true_answer)
                if 0.90 < ratio < 1.10:
                    score += 1.5
                elif 0.80 < ratio < 1.20:
                    score += 1.0
                else:
                    score -= 1.5
            except:
                score -= 1.5

        scores.append(score)

    return scores


In [ ]:
match_numbers = re.compile(
    solution_start + r".*?([\d\.\,]{1,})",
    flags = re.MULTILINE | re.DOTALL
)
print(match_numbers.findall("<SOLUTION>  0.34  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>  123,456  </SOLUTION>"))

['0.34']
['123,456']


In [ ]:
global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses=[]

    for r in responses:
      if(match_numbers.search(r) is not None):
        guess = match_numbers.search(r)
        extracted_responses.append(guess.group(1))
      else:
        extracted_responses.append(None)


    scores = []
    # Print only every few steps
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1

    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(-2.5)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            # Remove commas like in 123,456
            guess       = float(guess.strip().replace(",", ""))
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            scores.append(0)
            continue
    return scores

In [ ]:
dataset[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': '72',
 'prompt': [{'content': [{'text': 'You are given a problem.\n  Think about the problem and provide your working out.\n  Place it between <start_working_out><end_working_out>\n  Then provide the solution.\n  Place it between <SOLUTION></SOLUTION>\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
     'type': 'text'}],
   'role': 'user'}]}

In [ ]:
max(dataset.map(
    lambda x: {"tokens" : tokeniser.apply_chat_template(x["prompt"], add_generation_prompt = True, tokenize = True)},
    batched = False,
).map(lambda x: {"length" : len(x["tokens"])})["length"])

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

1

In [ ]:
max_prompt_length = 500 + 1 # + 1 just in case!

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 50,
    max_grad_norm = 1.0,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokeniser,
    reward_funcs = [
        match_format_exactly,
        match_format_partially,
        check_answer,
        check_numbers,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 91,619,328 of 5,531,057,600 (1.66% trained)
`generation_config` default values have been modified to match model-specific defaults: {'top_p': 0.95}. If this is not desired, please set these values explicitly.


********************Question:
[{'text': 'You are given a problem.\n  Think about the problem and provide your working out.\n  Place it between <start_working_out><end_working_out>\n  Then provide the solution.\n  Place it between <SOLUTION></SOLUTION>\nA concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all?', 'type': 'text'}] 
Answer:
476 
Response:
<start_working_out>
First, we need to determine how many tickets Mr. Benson bought that exceeded 10. He bought 12 tickets, so 12 - 10 = 2 tickets exceeded 10.

Next, we calculate the discount for each of these 2 tickets. The discount is 5% of $40, which is 0.05 * $40 = $2.

So, the discount for the 2 tickets is 2 * $2 = $4.

The total cost of the 12 tickets without the discount is 12 * $40 = $480.

Now, we subtract the total discount from the original price: $480 - $4 = $476.

Therefore, Mr. Benson paid $476 in all.
</start_working_out>


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_partially / mean,rewards / match_format_partially / std,rewards / check_answer / mean,rewards / check_answer / std,rewards / check_numbers / mean,rewards / check_numbers / std
1,0.000000,0.000000,5.180090,653.250000,165.000000,1547.000000,0.250000,355.333344,165.000000,703.000000,0.000000,0.000000,0.000000,-0.250000,1.500000,0.750000,1.936492,-0.500000,2.708013
2,0.000000,-2.500000,1.224745,1335.250000,901.000000,1547.000000,0.500000,1123.500000,901.000000,1346.000000,0.043925,0.000000,0.000000,0.125000,0.750000,-1.500000,0.000000,-1.125000,0.750000
3,60092660231510998122496.000000,7.000000,0.000000,223.750000,185.000000,261.000000,0.000000,223.750000,185.000000,261.000000,60092655998127348394229760.000000,0.000000,0.000000,0.500000,0.000000,3.000000,0.000000,3.500000,0.000000
4,16104926454787610217807872.000000,-2.500000,0.000000,353.250000,253.000000,404.000000,0.000000,353.250000,253.000000,404.000000,16104925707694475232571031552.000000,0.000000,0.000000,0.500000,0.000000,-1.500000,0.000000,-1.500000,0.000000
5,337537658369608920334336.000000,-2.500000,0.000000,152.750000,146.000000,158.000000,0.000000,152.750000,146.000000,158.000000,337537642228707855838478336.000000,0.000000,0.000000,0.500000,0.000000,-1.500000,0.000000,-1.500000,0.000000
6,4776616051347415791304704.000000,-0.125000,4.750000,491.000000,361.000000,630.000000,0.000000,491.000000,361.000000,630.000000,4776615746976138575097102336.000000,0.000000,0.000000,0.500000,0.000000,-0.375000,2.250000,-0.250000,2.500000
7,131749798694172776988672.000000,2.250000,3.752777,601.000000,499.000000,670.000000,0.000000,601.000000,499.000000,670.000000,131749790767837432816599040.000000,0.000000,0.000000,0.500000,0.000000,0.000000,1.732051,1.750000,2.020726
8,1628060976181130821632.000000,-3.500000,2.000000,296.750000,196.000000,437.000000,0.000000,296.750000,196.000000,437.000000,1628060932059928222236672.000000,0.000000,0.000000,-0.625000,2.250000,-1.125000,0.750000,-1.750000,0.500000
9,38160165125035655168.000000,-1.250000,2.500000,572.000000,212.000000,1547.000000,0.250000,247.000000,212.000000,271.000000,38160166426857422454784.000000,0.000000,0.000000,-0.250000,1.500000,0.750000,0.500000,-1.750000,0.500000
10,129715508809478176768.000000,3.500000,4.000000,460.500000,232.000000,592.000000,0.000000,460.500000,232.000000,592.000000,129715497972691573407744.000000,0.000000,0.000000,0.500000,0.000000,0.750000,1.500000,2.250000,2.500000


********************Question:
[{'text': 'You are given a problem.\n  Think about the problem and provide your working out.\n  Place it between <start_working_out><end_working_out>\n  Then provide the solution.\n  Place it between <SOLUTION></SOLUTION>\nRene can finish reading 30 pages in 60 minutes. Lulu can read 27 pages in 60 minutes and Cherry can read 25 pages in 60 minutes. If they have been reading for 240 minutes now, how many pages have they finished reading in total?', 'type': 'text'}] 
Answer:
328 
Response:
<start_working_out>
First, we need to find out how many pages each person has read in the 240 minutes.

Rene reads 30 pages in 60 minutes, so in 240 minutes, Rene reads (30 pages / 60 minutes) * 240 minutes = 30 * 4 = 120 pages.

Lulu reads 27 pages in 60 minutes, so in 240 minutes, Lulu reads (27 pages / 60 minutes) * 240 minutes = 27 * 4 = 108 pages.

Cherry reads 25 pages in 60 minutes, so in 240 minutes, Cherry reads (25 pages / 60 minutes) * 240 minutes = 25 * 4 = 10

TrainOutput(global_step=100, training_loss=5.324713173959623e+24, metrics={'train_runtime': 8923.856, 'train_samples_per_second': 0.045, 'train_steps_per_second': 0.011, 'total_flos': 0.0, 'train_loss': 5.324713173959623e+24})

In [ ]:
model.to(torch.bfloat16)      # <--- VERY IMPORTANT
model.to(model.device)

text = tokeniser.apply_chat_template(
    [
        {"role": "user", "content": [{"type": "text", "text": "What is the best way for a new fitness enthusiast to increase VO2 max?"}]},
    ],
    tokenize=False,
    add_generation_prompt=True
)

raw = tokeniser(text=text, return_tensors="pt")

inputs = {}
for k, v in raw.items():
    if v.dtype.is_floating_point:
        # floating tensors → match model dtype
        inputs[k] = v.to(model.device).to(model.dtype)
    else:
        # int tensors stay int
        inputs[k] = v.to(model.device)

output = model.fast_generate(**inputs,max_new_tokens=1024)


In [ ]:
decoded = tokeniser.decode(output[0], skip_special_tokens=True)
print(decoded)


user
What is the best way for a new fitness enthusiast to increase VO2 max?
model
Okay, let's break down the best ways for a new fitness enthusiast to increase their VO2 max.  It's a fantastic goal, and it's achievable with a structured approach. Here's a comprehensive guide, covering everything from foundational principles to specific exercises and important considerations.

**Understanding VO2 Max**

First, let's clarify what VO2 max *is*. VO2 max (maximal oxygen uptake) is the maximum amount of oxygen your body can use during intense exercise.  It's a key indicator of cardiovascular fitness and overall aerobic capacity.  Increasing VO2 max translates to improved endurance, speed, and overall athletic performance.

**Important Disclaimer:** *Before starting any new exercise program, consult with your doctor, especially if you have any underlying health conditions.*

**Here's a breakdown of the best strategies, broken down into categories:**

**1. Foundational Principles (The Building

In [ ]:
"model.save_pretrained("grpo_saved_lora")

In [ ]:
# Merge to 16bit
if True: model.save_pretrained_merged("Gemma3n_reasoning", tokeniser, save_method = "merged_16bit",)
if True: model.push_to_hub_merged("RachitD15673/Gemma3n_reasoning", tokeniser, save_method = "merged_16bit", token = "")

# # Merge to 4bit
# if False: model.save_pretrained_merged("Gemma3n_reasoning_4bit", tokenizer, save_method = "merged_4bit",)
# if False: model.push_to_hub_merged("hf/Gemma3n_reasoning", tokenizer, save_method = "merged_4bit", token = "")

# # Just LoRA adapters
# if False:
#     model.save_pretrained("Gemma3n_reasoning_LORA")
#     tokenizer.save_pretrained("Gemma3n_reasoning_LORA")
# if False:
#     model.push_to_hub("hf/Gemma3n_reasoning", token = "")
#     tokenizer.push_to_hub("hf/Gemma3n_reasoning", token = """)


Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 3 files from cache to `Gemma3n_reasoning`: 100%|██████████| 3/3 [00:28<00:00,  9.38s/it]


Successfully copied all 3 files from cache to `Gemma3n_reasoning`
Checking cache directory for required files...


Unsloth: Copying 1 files from cache to `Gemma3n_reasoning`: 100%|██████████| 1/1 [00:00<00:00, 23.44it/s]


Successfully copied all 1 files from cache to `Gemma3n_reasoning`


...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0003-of-00003.safetensors:   0%|          | 10.2kB / 2.82GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 3/3 [03:13<00:00, 64.50s/it]


Unsloth: Merge process complete. Saved to `/content/RachitD15673/Gemma3n_reasoning`
